# Table of Contents
0. Python/SQL setup
1. Database exploration and querying data
2. Exporting dataframes

0 -- Python/SQL Setup

In [1]:
# 0.0 -- Import packages
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime, date
import regex as re

In [2]:
# 0.1 -- Check pyodbc drivers
for driver in pyodbc.drivers():
    print(driver)

SQL Server
PostgreSQL ANSI(x64)
PostgreSQL Unicode(x64)
MySQL ODBC 8.0 ANSI Driver
MySQL ODBC 8.0 Unicode Driver
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [3]:
# 0.2 -- Define our server name and database name
# And establish a connection to the database
server = "SLIME-EXE\SQLEXPRESS"
database = "AdventureWorks2019"
cnxn = pyodbc.connect(f"""
DRIVER={{ODBC Driver 17 for SQL SERVER}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
MARS_CONNECTION=yes;
""")

In [4]:
# 0.3 -- Create the connection cursor
cursor = cnxn.cursor()

### 1 -- Begin exploring database and querying data

Logic of table joins used in the following query:

| Table | Alias |Reason | Key |
| :--------- | :--------- | :--------- | :--------- |
| 1. Sales.SalesOrderHeader | header |Location/sales data | CustomerID, SalesOrderID, ShipToAddressID
| 2. Person.Address | address | Location data at a more granular level | AddressID, StateProvinceID
| 3. Person.StateProvince | state |State data | StateProvinceID, CountryRegionCode
| 4. Person.CountryRegion | country |Country data | CountryRegionCode  
| 5. Sales.vPersonDemographics | demo | Demographic data per customer | BusinessEntityID
| 6. Person.BusinessEntityAddress | ent | PK BusinessEntityID | AddressID, BusinessEntityID
| 7. Sales.SalesOrderDetail | detail | Numerical sales data | ProductID, SalesOrderID
| 8. Production.Product | prod | Categorical/dimensional data regarding products | ProductID, ProductSubcategoryID
| 9. Production.ProductSubcategory | subcat | Subcategory of products | ProductSubcategoryID, ProductCategoryID
| 10. Production.ProductCategory| cat | Category of products | ProductCategoryID


In [5]:
# 1.1 -- Collecting demographic data from customers in North America
query = """SELECT demo.BusinessEntityID, 
                state.Name, address.StateProvinceID, state.StateProvinceCode, 
                CAST(BirthDate AS date) AS BirthDate, MaritalStatus, YearlyIncome, Gender, TotalChildren, 
                NumberCarsOwned, Education, Occupation, City, 
                state.StateProvinceCode AS StateCode, state.Name AS State, 
                state.CountryRegionCode, country.Name AS Country, address.PostalCode, 
                cat.Name AS Category, subcat.Name AS Subcategory,  
                SUM(detail.OrderQty) AS OrderQty, SUM(header.TotalDue) AS TotalDue 
        FROM Sales.SalesOrderHeader AS header 
        FULL OUTER JOIN Person.Address AS address 
        ON address.AddressID = header.ShipToAddressID 
        FULL OUTER JOIN Person.StateProvince AS state 
        ON state.StateProvinceID = address.StateProvinceID 
        FULL OUTER JOIN Person.CountryRegion AS country 
        ON country.CountryRegionCode = state.CountryRegionCode 
        FULL OUTER JOIN Sales.vPersonDemographics AS demo 
        ON demo.BusinessEntityID = header.CustomerID 
        FULL OUTER JOIN Person.BusinessEntityAddress AS ent
        ON ent.BusinessEntityID = demo.BusinessEntityID 
        INNER JOIN Sales.SalesOrderDetail AS detail
        ON detail.SalesOrderID = header.SalesOrderID 
        INNER JOIN Production.Product AS prod
        ON prod.ProductID = detail.ProductID 
        INNER JOIN Production.ProductSubcategory AS subcat
        ON subcat.ProductSubcategoryID = prod.ProductSubcategoryID 
        INNER JOIN Production.ProductCategory AS cat
        ON cat.ProductCategoryID = subcat.ProductCategoryID 
        GROUP BY demo.BusinessEntityID, 
                state.Name, address.StateProvinceID, state.StateProvinceCode, 
                CAST(BirthDate AS date), MaritalStatus, YearlyIncome, Gender, TotalChildren, 
                NumberCarsOwned, Education, Occupation, City, 
                state.StateProvinceCode, state.Name, state.CountryRegionCode, 
                country.Name, address.PostalCode, 
                cat.Name, subcat.Name 
        ORDER BY state.Name"""
custdf = pd.read_sql(query, cnxn) 
print(custdf.shape)
display(custdf.head())

(35201, 22)


,BusinessEntityID,Name,StateProvinceID,StateProvinceCode,BirthDate,MaritalStatus,YearlyIncome,Gender,TotalChildren,NumberCarsOwned,...,City,StateCode,State,CountryRegionCode,Country,PostalCode,Category,Subcategory,OrderQty,TotalDue
0,NaN,Alabama,3,AL,None,None,None,None,NaN,NaN,...,Birmingham,AL,Alabama,US,United States,35203,Bikes,Mountain Bikes,28,66276.2513
1,NaN,Alabama,3,AL,None,None,None,None,NaN,NaN,...,Birmingham,AL,Alabama,US,United States,35203,Clothing,Shorts,12,7971.1882
2,NaN,Alabama,3,AL,None,None,None,None,NaN,NaN,...,Birmingham,AL,Alabama,US,United States,35203,Components,Bottom Brackets,3,4970.6621
3,NaN,Alabama,3,AL,None,None,None,None,NaN,NaN,...,Birmingham,AL,Alabama,US,United States,35203,Components,Brakes,1,4888.3283
4,NaN,Alabama,3,AL,None,None,None,None,NaN,NaN,...,Birmingham,AL,Alabama,US,United States,35203,Components,Cranksets,3,7971.1882


In [6]:
# 1.2 -- Checking the status of our NaNs in custdf
# NaNs in our demographic data is just fine
# We're more interested in our TotalDue column, which has 0 NaNs
def nanpct(x) :
    print(x.shape)
    dfnan = x.isna().sum()
    dfnanpct = round(x.isna().sum() / len(x) * 100, 1)     
    display(pd.DataFrame(pd.concat((dfnan, dfnanpct), axis = 1).rename(columns= {0: 'NaNs', 1: '%total'})))
nanpct(custdf)

(35201, 22)


,NaNs,%total
BusinessEntityID,7645,21.7
Name,0,0.0
StateProvinceID,0,0.0
StateProvinceCode,0,0.0
BirthDate,7645,21.7
MaritalStatus,7645,21.7
YearlyIncome,7645,21.7
Gender,7645,21.7
TotalChildren,7645,21.7
NumberCarsOwned,7645,21.7


In [7]:
# 1.3 Next, we'll create an Age column by subtracting BirthDate from today's date
custdf.dtypes

BusinessEntityID     float64
Name                  object
StateProvinceID        int64
StateProvinceCode     object
BirthDate             object
MaritalStatus         object
YearlyIncome          object
Gender                object
TotalChildren        float64
NumberCarsOwned      float64
Education             object
Occupation            object
City                  object
StateCode             object
State                 object
CountryRegionCode     object
Country               object
PostalCode            object
Category              object
Subcategory           object
OrderQty               int64
TotalDue             float64
dtype: object

In [8]:
# 1.4 -- BirthDate will first need to be converted to datetime
custdf['BirthDate'] = pd.to_datetime(custdf['BirthDate'])

In [9]:
# 1.5 -- Now, we're writing a function and applying lambda to create our new Age column
# We'll subtract 5yrs from Age to account for an approximate difference in the present year and the year of recording
# Age will display values of customer ages closer to date of purchase
def calculate_age(x):
    today = date.today()
    return today.year - x.year - ((today.month, today.day) < (x.month, x.day))
custdf['Age'] = custdf.BirthDate.apply(lambda x: calculate_age(x) - 5)

In [10]:
# 1.6 -- Dropping BirthDate and rearranging columns
custdf.drop(columns= 'BirthDate', inplace= True)
custdf = custdf [['BusinessEntityID', 'StateProvinceID', 'StateProvinceCode', 'Age',
       'MaritalStatus', 'YearlyIncome', 'Gender', 'TotalChildren',
       'NumberCarsOwned', 'Education', 'Occupation', 'City', 'StateCode',
       'State', 'CountryRegionCode', 'Country', 'PostalCode', 'Category', 'Subcategory', 'OrderQty',
       'TotalDue']]

In [11]:
# Checkpoint: all set with custdf
custdf.sample(5)

,BusinessEntityID,StateProvinceID,StateProvinceCode,Age,MaritalStatus,YearlyIncome,Gender,TotalChildren,NumberCarsOwned,Education,...,City,StateCode,State,CountryRegionCode,Country,PostalCode,Category,Subcategory,OrderQty,TotalDue
16869,NaN,19,HE,NaN,None,None,None,NaN,NaN,None,...,Darmstadt,HE,Hessen,DE,Germany,64283,Accessories,Cleaners,1,829.0815
1646,12077.0,7,BC,56.0,S,25001-50000,F,3.0,0.0,Graduate Degree,...,Shawnee,BC,British Columbia,CA,Canada,V9B 5T2,Accessories,Tires and Tubes,4,199.6516
24693,19804.0,58,OR,65.0,M,greater than 100000,F,0.0,4.0,Partial College,...,Corvallis,OR,Oregon,US,United States,97330,Clothing,Jerseys,1,136.9979
12885,12648.0,14,ENG,60.0,M,75001-100000,M,1.0,1.0,Partial College,...,Warrington,ENG,England,GB,United Kingdom,WA1,Accessories,Tires and Tubes,2,5398.0134
9850,18505.0,9,CA,42.0,S,25001-50000,F,3.0,2.0,Partial College,...,San Francisco,CA,California,US,United States,94109,Accessories,Bottles and Cages,1,15.4479


Creating the Correlation Matrix

In [ ]:
# In order to create a correlation matrix in Tableau, we need a single column

2 -- Exporting dataframes

In [31]:
# 2.0 -- Our dataframe is now ready for export
custdf.to_csv(path_or_buf= 'C:\\Users\\Bruen\\Dropbox\\My PC (DESKTOP-V41C1LD)\\Desktop\\AdventureWorksCustDF.csv')